In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import ibm_db
import ibm_db_sa
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy.sql import text as sa_text
import pyodbc
from minio import Minio
import urllib3
from io import BytesIO

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
os.chdir('C:\\Users\\spiegez\\OneDrive - Memorial Sloan Kettering Cancer Center\\Github\\Research')
logins = pd.read_csv('logins.csv')

In [3]:
ACCESS_KEY = logins.username[2]
SECRET_KEY = logins.pw[2]


httpClient = urllib3.PoolManager(
                cert_reqs='CERT_REQUIRED',
                ca_certs='certificate.crt'
            )

client = Minio(
    "tllihpcmind6:9000",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure=True,
    http_client=httpClient
)

In [4]:
res = client.get_object('acir',"CURES/CURES_active_patient")

pt_act = pd.read_parquet(BytesIO(res.data))

In [5]:
res1 = client.get_object('acir',"CURES/CURES_portal_active_patient")

port_act = pd.read_parquet(BytesIO(res1.data))

In [6]:
res3 = client.get_object('acir',"CURES/CURES_portal_reports_read")

reports = pd.read_parquet(BytesIO(res3.data))

In [7]:
res4 = client.get_object('acir',"CURES/CURES_portal_psms_sent")

psms = pd.read_parquet(BytesIO(res4.data))

In [14]:
res5 = client.get_object('acir',"CURES/CURES_demographic")

demo = pd.read_parquet(BytesIO(res5.data))

In [15]:
res6 = client.get_object('acir',"CURES/CURES_account_created")

act_cr = pd.read_parquet(BytesIO(res6.data))

In [17]:
pt_act['mrn'] =  pd.to_numeric(pt_act['mrn'], errors='coerce')
port_act['mrn'] =  pd.to_numeric(port_act['mrn'], errors='coerce')
psms['mrn'] =  pd.to_numeric(psms['mrn'], errors='coerce')
reports['mrn'] =  pd.to_numeric(reports['mrn'], errors='coerce')

demo['mrn'] =  pd.to_numeric(demo['mrn'], errors='coerce')
act_cr['mrn'] =  pd.to_numeric(act_cr['mrn'], errors='coerce')
all_reps['mrn'] =  pd.to_numeric(all_reps['mrn'], errors='coerce')

In [18]:
act = pt_act.merge(port_act, on=['mrn','week'],how='left')

#because i cannot figure out the formatting of those mrns (i.e. trim then drop nulls dosent work)
act = act.dropna(subset=['mrn']).reset_index(drop=True)

In [23]:
df1 = act.merge(psms, on=['mrn','week'],how='left')
df1 = df1.merge(reports, on=['mrn','week'],how='left')

df1 = df1.drop_duplicates().reset_index(drop=True)
df1 = df1.dropna(subset=['mrn']).reset_index(drop=True)

In [24]:
os.chdir('C:\\Users\\spiegez\\OneDrive - Memorial Sloan Kettering Cancer Center\\Documents\\Research\\CURES')
year_adj = pd.read_csv('CURES_biostats_year_adjust_table.csv')

year_adj['week_text'] = year_adj['start date'].astype(str) + ' - ' + year_adj['end date'].astype(str) 
year_adj = year_adj.drop(columns=['start date', 'start doy', 'year', 'end date', 'end doy', 'year.1'])

In [26]:
dfa = df1.merge(all_reps, on=['mrn','week'],how='left')
df = dfa.merge(year_adj, on = 'week', how='left')

df['ActiveUserFlag'] = df['ActiveUserFlag'].fillna('N')
df['activepatientflag'] = df['activepatientflag'].fillna('N')
df['NumLoginDays'] = df['NumLoginDays'].fillna(0)
df['NumPSMsSent'] = df['NumPSMsSent'].fillna(0)
df['NumReportsRead'] = df['NumReportsRead'].fillna(0)
df['NumReportsPosted'] = df['NumReportsPosted'].fillna(0)

df = df.drop_duplicates().reset_index(drop=True)

In [27]:
##QC

In [28]:
len(df.index)

1372000

In [29]:
print(df.loc[((df['ActiveUserFlag'] == 'N') & (df['NumLoginDays']> 0))])

Empty DataFrame
Columns: [mrn, week, activepatientflag, ActiveUserFlag, NumLoginDays, NumPSMsSent, NumReportsRead, NumReportsPosted, week_text]
Index: []


In [18]:
print(df.loc[((df['ActiveUserFlag'] == 'N') & (df['NumReportsRead']> 0))])

Empty DataFrame
Columns: [mrn, week, activepatientflag, ActiveUserFlag, NumLoginDays, NumPSMsSent, NumReportsRead, week_text]
Index: []


In [19]:
print(df.loc[((df['ActiveUserFlag'] == 'Y') & (df['NumLoginDays'] == 0))])

Empty DataFrame
Columns: [mrn, week, activepatientflag, ActiveUserFlag, NumLoginDays, NumPSMsSent, NumReportsRead, week_text]
Index: []


In [24]:
df.to_csv('biostats_CURES_weekly_act_dataset.csv')

In [21]:
mrns = act['mrn'].to_frame()
mrns = mrns.drop_duplicates().reset_index(drop=True)

In [22]:
demo_fin = mrns.merge(demo, on=['mrn'],how='left')
demo_fin = demo_fin.merge(act_cr, on=['mrn'],how='left')

In [23]:
demo_fin.to_csv('biostats_CURES_demographic_dataset.csv')